In [1]:
!pip install rank_bm25 pypdf

  Using cached rank_bm25-0.2.2-py3-none-any.whl.metadata (3.2 kB)
Using cached rank_bm25-0.2.2-py3-none-any.whl (8.6 kB)
   ---------------------------------------- 0.0/302.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/302.3 kB ? eta -:--:--
   ----- --------------------------------- 41.0/302.3 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 302.3/302.3 kB 3.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [3]:
from langchain_postgres import PGVector
from langchain_openai import OpenAIEmbeddings
import os 
embeddings = OpenAIEmbeddings()
vectorstore = PGVector(
    embeddings=embeddings,
    collection_name="chapter6_collection",
    connection=f"postgresql://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@{os.getenv('PGVECTOR_HOST')}:{os.getenv('PGVECTOR_PORT')}/{os.getenv('POSTGRES_DB')}",
    use_jsonb=True
)

## 다양한 쿼리 & Ensemble Retriever

In [4]:
from langchain.retrievers.multi_query import MultiQueryRetriever

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 10}),
    llm=llm
)
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [5]:

docs = retriever_from_llm.get_relevant_documents("연회비 반환조건")
print(docs)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_12904\3554555831.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever_from_llm.get_relevant_documents("연회비 반환조건")
INFO:langchain.retrievers.multi_query:Generated queries: ['연회비 환불 조건은 무엇인가요?  ', '연회비를 돌려받기 위한 조건은 어떻게 되나요?  ', '연회비를 반환받기 위해 필요한 조건은 무엇인가요?']


[Document(id='235d3619-095e-4809-af8d-d5f1df5eb540', metadata={'page': 4, 'title': '', 'author': '임재봉', 'format': 'PDF 1.5', 'source': 'data/baro_rewardsplus_card.pdf', 'creator': 'Microsoft® Word 2016', 'modDate': "D:20220926093210+09'00'", 'moddate': '2022-09-26T09:32:10+09:00', 'subject': '', 'trapped': '', 'keywords': '', 'producer': 'Microsoft® Word 2016', 'file_path': 'data/baro_rewardsplus_card.pdf', 'total_pages': 10, 'creationDate': "D:20220926093210+09'00'", 'creationdate': '2022-09-26T09:32:10+09:00'}, page_content='신용카드 상품별로 상이 \n(해당 상품의 연회비는 상품안내장 참고) \n제휴연회비  \n카드별 \n \n□ 연회비 청구 : 카드사는 카드이용대금에 우선하여 연회비를 청구할 수 있으며, 연회비는 신용카드 \n발급 시점을 기준으로 원칙적으로 1년 단위로 청구됩니다. 다만, 연회비 부과 시점을 기준으로 1년 \n이상 사용하지 않은 카드에 대한 연회비는 청구하지 않습니다. \n \n□ 연회비 반환 : 회원이 유효기간이 도래하기 전에 카드를 해지하는 경우 연회비 반환금액은 회원이'), Document(id='5d33eea4-c6cd-4722-83db-47bc97501494', metadata={'page': 5, 'title': '', 'author': '임재봉', 'format': 'PDF 1.5', 'source': 'data/baro_rewardsplus_card.pdf', 'creator': 'Microsoft® Word 20

In [6]:
for doc in docs:
    print(doc.page_content)
    print("-"*100)

신용카드 상품별로 상이 
(해당 상품의 연회비는 상품안내장 참고) 
제휴연회비  
카드별 
 
□ 연회비 청구 : 카드사는 카드이용대금에 우선하여 연회비를 청구할 수 있으며, 연회비는 신용카드 
발급 시점을 기준으로 원칙적으로 1년 단위로 청구됩니다. 다만, 연회비 부과 시점을 기준으로 1년 
이상 사용하지 않은 카드에 대한 연회비는 청구하지 않습니다. 
 
□ 연회비 반환 : 회원이 유효기간이 도래하기 전에 카드를 해지하는 경우 연회비 반환금액은 회원이
----------------------------------------------------------------------------------------------------
참고해주시기 바랍니다. 
 
11  
연체이자율 및 연체에 따른 불이익 
 
□  연체이자율은 [회원별·이용상품별 약정이자율 + 연체가산이자율]로 적용합니다. 
ㅇ 연체가산이자율은 연 3%로 최고 연 20%로 합니다.  
(예시) * 연회비 2만원, 반환금액 산정 제외비용 1만원 
* 카드등록일 : 2021.03.02일, 카드탈회일 : 2021.05.10일 
 반환연회비 = (2만원 – 1만원) X (연회비만기도래일:’22.03.01-‘21.05.10:295일) / 365일 : 8,082원 
※ 윤년인 경우 1일 추가 계산하여 금액 산정함.
----------------------------------------------------------------------------------------------------
Q2. 카드 중도해지 시 연회비는 어떻게 반환받나요? 
→ 연회비는 카드발급 시점을 기준으로 원칙적으로 1년 단위로 청구됩니다. 연회비 부과기간이 끝나기 이전에 카드를  
중도해지 하는 경우 남은 기간에 해당하는 연회비를 계산하여 10영업일(불가피한 경우 3개월)이내에 반환하여 드립니다. 
다만, 카드발급 및 부가서비스 제공에 이미 지출된 비용은 제외됩니다.  
 
Q3. 일부결제금액이월약정(리볼빙) 이용 시 주의사

In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


loader = PyPDFLoader('data/baro_rewardsplus_card.pdf')

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_docs = loader.load_and_split(text_splitter=text_splitter)


In [13]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(split_docs)
bm25_retriever.k = 2

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vectorstore.as_retriever()], weights=[1, 0.5]
)

In [14]:
docs = ensemble_retriever.invoke("연회비 반환조건")


In [15]:
for doc in docs:
    print(doc.page_content)
    print("-"*100)


(해당 상품의 연회비는 상품안내장 참고) 제휴연회비  카드별 
 
□ 연회비 청구 : 카드사는 카드이용대금에 우선하여 연회비를 청구할 수 있으며, 연회비는 신용카드 
발급 시점을 기준으로 원칙적으로 1년 단위로 청구됩니다. 다만, 연회비 부과 시점을 기준으로 1년 
이상 사용하지 않은 카드에 대한 연회비는 청구하지 않습니다. 
 
□ 연회비 반환 : 회원이 유효기간이 도래하기 전에 카드를 해지하는 경우 연회비 반환금액은 회원이
----------------------------------------------------------------------------------------------------
할부결제 할부기간에 걸쳐 동일 금액을 분할하여 결제하는 방식 연11%~18.5% 
단기카드대출 
(현금서비스) 
카드사가 미리 부여한 한도(신용카드 이용한도의  
최대 40%) 이내에서 이용할 수 있는 단기 금융서비스 
※ 이용 후 최초로 도래하는 결제일에 상환의무 발생 
연7.85%~19.9% 
일부결제금액 
이월약정(리볼빙) 
신용카드 이용대금 중 일정금액 이상만 결제하면 잔여 
대금에 대한 상환이 자동으로 연장되고 잔여 이용한도 
내에서는 신용카드를 계속 이용할 수 있는 결제 방식 
연7.90%~ 19.9% 
□ 금리의 변동 여부 
ㅇ 회원이 할부결제, 단기카드대출(현금서비스) 및 일부결제금액이월약정(리볼빙) 이용 시 이용개월, 
이용실적 및 신용도 등에 따라 금리는 변동 가능합니다. 
 
5  수수료율 
 
□ 해외서비스 수수료 : 신용카드 해외 이용 시(해외 사이트 거래 포함) 부과하는 수수료 
ㅇ 수수료율 : ( 0 ) % 
- (거래금액 × 해외서비스 수수료율) × 전신환매도율 
 
□ 국제브랜드 수수료 : 마스터/비자 등 국제카드 브랜드사가 카드사에 부과하는 수수료 
ㅇ 수수료율 : ( 0.8 ~ 1.1 ) %  
- (거래금액 × 국제브랜드사 이용수수료율) × 전신환매도율 
 
□ 자동화기기(ATM) 이용수수료 : 단기카드대출(현금서비스)을 자